### Visualizing Volume Fraction Profiles from SF-SCF Calculations

We employ a two-gradient Scheutjens-Fleer Self-Consistent Field (SF-SCF) method in cylindrical geometry.  
A full description is provided in Supplementary Note 3 of the original work.  
Here we present technical details for the purposes of reproducibility.  

We do not discuss the generation of input files, as this is extensively covered in the Namics/SFbox manual.  
However, we provide an example below.  

Each grafted chain is anchored by a monomer segment pinned to the pore surface.  
Pinned monomers are constrained to the inner cylindrical surface (constant radius).  
This allows them to slide freely along the **z-axis** (one less degree of freedom than a free segment).  
To prevent such lateral motion, the pore is divided along its **z-axis** into discrete slices (rings).  
Each slice defines its own grafting site, thereby immobilizing pinned segments within that slice.  
For each slice, a fraction of chains is grafted to the wall, so that the total polymer quantity is distributed evenly along the pore length and the grafting density is conserved.  

Example input snippet from `SCF/empty_pore_input.in`:

```text
mol:pol0:composition:(P0)1(P)299
mol:pol0:freedom:restricted
mol:pol0:theta:980.1769079200155
mon:P0:freedom:pinned
mon:P0:pinned_range:26,121;26,121
...
mol:pol51:composition:(P0)1(P)299
mol:pol51:freedom:restricted
mol:pol51:theta:980.1769079200155
mon:P51:freedom:pinned
mon:P51:pinned_range:26,172;26,172
```

The numerical SF-SCF algorithm then minimizes the excess Helmholtz free energy subject to the incompressibility constraint, yielding the **equilibrium volume fraction distribution** of polymer segments originating from each grafting slice.  
- In our system, the pore has a length of 52 lattice units, with slices labeled from $z = -26$ to z = $+25$.  
- Each slice corresponds to one set of grafted chains, allowing us to examine how different grafting positions contribute to the overall polymer distribution.  
- The results are stored in the Namics/SFbox output files, which provide the local volume fractions $\phi(r,z)$ for every grafting site.  


The code below loads these results, and plots the **polymer volume fraction profiles**  with an interactive slider.  
An interactive **slider** lets you scroll through the slices along the $z$-axis, visualizing how chains grafted at different axial positions distribute inside the pore.

In [1]:
%matplotlib ipympl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from src.read_namics_output import extract_volume_fractions_per_grafting_range, create_walls
from src.utils import mirror_extend
from src.make_plot import plot_with_slider, plot_grid

raw = pd.read_table("SCF/output/empty_pore_input.pro", index_col=False)
R, Z, Phis = extract_volume_fractions_per_grafting_range(raw)
pore_radius = 26
pore_length = 52
walls = create_walls(len(R), len(Z), pore_radius, pore_length)
fig, ax = plot_with_slider(
    Z, 
    np.concatenate((-R[::-1], R)), 
    #for readability of the plot and consistent formatting with the original work we plot the mirrored array along z axis
    mirror_extend(Phis.transpose(0,2,1), axis=1), 
    fix_clim=True, 
    walls = mirror_extend(walls.T, axis = 0),
    xlim = (-50,50),
    ylim = (-50,50)
)

In [2]:
phi = np.sum(Phis, axis = 0)
fig, ax = plot_grid(
    Z, 
    np.concatenate((-R[::-1], R)), 
    mirror_extend(phi.T, axis=0), 
    walls = mirror_extend(walls.T, axis = 0),
    xlim = (-50,50),
    ylim = (-50,50)
)